# TODO:
### 24Oct23 - 31Oct23

In [1]:
from CReSIS_project_classes import *
from functions import *
from iceflow_library import *
from CReSIS_project_classes import *

In [2]:
zoom = True
seg_length = 100
season = "2018_Antarctica_DC8"
# season = "2016_Antarctica_DC8"
flight = "20181030_01"  # the flight date and frame number
# flight = "20181103_01"
# flight = "20181112_02"  # the problem flight
# flight = "20161024_05"
file_name = "layer_export_" + flight + ".pickle"

### read in the layers from the layer files and save them to a pickle file

In [12]:
# if the file at filename exists, read in the layers from the pickle file
# otherwise, read in the layers from the layer files and save them to a pickle file
if not os.path.isfile(file_name):  # if the file does not exist
    # file_name = "layer_export_" + flight + ".pickle"
    print(f"File {file_name} does not exist. Making it...")
    mat_pickler(season, flight, testing_mode=True)  # make it
    
layers = read_layers(file_name)  # read in the layers from the pickle file

Reading pickle file...
--------------------
Surface
Bottom
Test
--------------------


### read in the iceflow data from the iceflow data files and save them to a pickle file

In [14]:
try:
    iceflow_data = iceflow_data_file_loader()
    print("The iceflow data pickle file was found and loaded.")
except FileNotFoundError:
    print("The iceflow data pickle file was not found. Creating a new one...")
    filename = iceflow_saver()
    iceflow_data = iceflow_loader(filename)
    print("The iceflow data pickle file was successfully created.")
    
x = iceflow_data[0]
y = iceflow_data[1]
velocity_x = iceflow_data[2]
velocity_y = iceflow_data[3]
latitude = iceflow_data[4]
longitude = iceflow_data[5]

The iceflow data pickle file was found and loaded.


# next step: continue working through crossover.py to rebuild the code